Imports und definition der Ordner. BASE_FOLDER ist der Ordner mit den Originaldateien, OUTPUT_FOLDER ist der Ordner, in den die Dateien kopiert werden sollen. `MAX_IMAGES_PER_CLASS` ist die maximale Anzahl von Bildern pro Klasse, die kopiert werden sollen. 

In [2]:
import os
import shutil
import nltk
from nltk.corpus import wordnet as wn
import random

nltk.download("wordnet")
BASE_FOLDER = "imagenet"
IMAGE_DATA_FOLDER = os.path.join(BASE_FOLDER, "ILSVRC", "Data", "CLS-LOC", "train")
MAX_IMAGES_PER_CLASS = 10000

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nijog\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Mit dieser Funktion können alle hyper oder hyponyme eines Synsets gefunden werden. Dies wird anschliessend verwendet um die Bilder in mehrere Klassen zu unterteilen.

In [3]:
# A hyponym is like a child in a tree, a more specific concept than the parent
def get_all_connected_synsets(synset, above=True):
    """
    Get all connected synsets of a synset. A connected synset is a synset that is either a hypernym or a hyponym of the
    given synset.
    :param synset: The synset to get the connected synsets of
    :param above: If True, get hypernyms. If False, get hyponyms
    :return: List of dictionaries containing information about the connected synsets
    """
    hypernyms = []
    if above:
        to_search = synset.hypernyms()
    else:
        to_search = synset.hyponyms()

    for hypernym in to_search:
        offset = hypernym.offset()
        wnid = "n{:08d}".format(offset)
        output = {
            "wnid": wnid,
            "name": hypernym.name(),
            "lemma_names": hypernym.lemma_names(),
            "definition": hypernym.definition(),
        }
        hypernyms.append(output)
        hypernyms.extend(get_all_connected_synsets(hypernym, above=above))

    return hypernyms

Hier wird mit dem `to_pick` Dictionary definiert, welche Klassen kopiert werden sollen. Es wird immer geschaut, ob die Klasse existiert und dann in den Output Ordner kopiert.

In [8]:
fish = wn.synsets("fish")[0]
bird = wn.synsets("bird")[0]
dog = wn.synsets("dog")[0]
snake = wn.synsets("snake")[0]
monkey = wn.synsets("monkey")[0]
lizard = wn.synsets("lizard")[0]

to_pick_1 = {
    "bird": get_all_connected_synsets(bird, above=False),
    "dog": get_all_connected_synsets(dog, above=False),
}
to_pick_2 = {
    "fish": get_all_connected_synsets(fish, above=False),
    "snake": get_all_connected_synsets(snake, above=False),
    "monkey": get_all_connected_synsets(monkey, above=False),
    "lizard": get_all_connected_synsets(lizard, above=False),
}
print(to_pick_1["bird"])

to_picks = [to_pick_1, to_pick_2]

[{'wnid': 'n01687665', 'name': 'agamid.n.01', 'lemma_names': ['agamid', 'agamid_lizard'], 'definition': 'a lizard of the family Agamidae'}, {'wnid': 'n01687978', 'name': 'agama.n.01', 'lemma_names': ['agama'], 'definition': 'small terrestrial lizard of warm regions of the Old World'}, {'wnid': 'n01688589', 'name': 'dragon.n.04', 'lemma_names': ['dragon', 'flying_dragon', 'flying_lizard'], 'definition': 'any of several small tropical Asian lizards capable of gliding by spreading winglike membranes on each side of the body'}, {'wnid': 'n01688243', 'name': 'frilled_lizard.n.01', 'lemma_names': ['frilled_lizard', 'Chlamydosaurus_kingi'], 'definition': 'large arboreal insectivorous Australian lizard with a ruff of skin around the neck'}, {'wnid': 'n01688961', 'name': 'moloch.n.03', 'lemma_names': ['moloch'], 'definition': 'any lizard of the genus Moloch'}, {'wnid': 'n01689081', 'name': 'mountain_devil.n.02', 'lemma_names': ['mountain_devil', 'spiny_lizard', 'Moloch_horridus'], 'definition':

In [ ]:
for i, to_pick in enumerate(to_picks):
    OUTPUT_FOLDER = f"synset_processed_{i}"
    image_counts = {category: 0 for category in to_pick.keys()}
    for category in to_pick.keys():
        for synset_dict in to_pick[category]:
            # check if images exist
            candidate_path = os.path.join(IMAGE_DATA_FOLDER, synset_dict["wnid"])
            if not os.path.exists(candidate_path):
                print(
                    f"Class {synset_dict['wnid']} ({category}) does not exist in the dataset, skipping...."
                )
                continue

            # Count images in the directory
            num_images = len(os.listdir(candidate_path))
            image_counts[category] += num_images

            # copy images
            images_dest = os.path.join(OUTPUT_FOLDER, "images", category)
            shutil.copytree(candidate_path, images_dest, dirs_exist_ok=True)
            print(
                f"Images for class {synset_dict['wnid']} ({category}) copied to output"
            )

    # Output the image counts for each category
    for category, count in image_counts.items():
        print(f"Total images for category {category}: {count}")

    # sample down
    for category in to_pick.keys():
        folder_path = os.path.join(OUTPUT_FOLDER, "images", category)
        if os.path.exists(folder_path):
            images = os.listdir(folder_path)
            if len(images) > MAX_IMAGES_PER_CLASS:
                # Randomly select images to delete
                to_delete = random.sample(images, len(images) - MAX_IMAGES_PER_CLASS)
                for image in to_delete:
                    os.remove(os.path.join(folder_path, image))

    # copy mapping files
    shutil.copy(
        os.path.join(BASE_FOLDER, "LOC_synset_mapping.txt"),
        os.path.join(OUTPUT_FOLDER, "LOC_synset_mapping.txt"),
    )
    # Copy localization files
    shutil.copy(
        os.path.join(BASE_FOLDER, "LOC_train_solution.csv"),
        os.path.join(OUTPUT_FOLDER, "LOC_train_solution.csv"),
    )
    print("Copied annotation files to output")

    print("Creating zip archive...")
    # create zip archive
    shutil.make_archive(OUTPUT_FOLDER, "zip", OUTPUT_FOLDER)
    # remove the output folder
    shutil.rmtree(OUTPUT_FOLDER)
    print("Done!")